In [1]:
import multiprocessing
import pandas as pd
from ckiptagger import WS
from gensim.models.doc2vec import TaggedDocument, Doc2Vec


# 讀取檔案
corpusDF=pd.read_csv("./lyricDF.csv")

In [2]:
# 斷詞

pth = "C:/Users/Admin/miniconda3"  # 請自行更改為斷詞data所在的資料位置
ws = WS(f"{pth}/data")

tokens = ws(corpusDF.Lyric)

tokenizedSong = []

for token in tokens:
    tokenizedSong.append(' '.join(token))

c:\Users\Admin\miniconda3\envs\pythonnlp2\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [3]:
# Doc2vec

             #看CPU有多少核心可以用
number_cors = multiprocessing.cpu_count()

corpus = tokenizedSong
training_corpus = []
for i, text in enumerate(corpus):

    tagged_doc = TaggedDocument(text, [i])               
    training_corpus.append(tagged_doc)

# 用 10 做為窗口大小及100做為詞向量的維度
# .min_count 是詞彙的最小文本頻率
# 建立模型
# size 改稱：vector_size；iter 改稱：epochs

model = Doc2Vec(vector_size=100, window=10, min_count=2, workers=number_cors, epochs=10)

# 訓練模型前先編輯詞彙
model.build_vocab(training_corpus)

# 進行10個世代的訓練
model.train(training_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
def recommand(query):

    # query斷詞
    inqTokens = ws(query)
    inqTokenized =' '.join(sum(inqTokens,[]))

    # 輸出結果
    print(f"輸入段落：{inqTokenized}")
    print()

    inferred_vector = model.infer_vector(doc_words=[inqTokenized], epochs=10)

    # dv = docvecs
    sims = model.docvecs.most_similar([inferred_vector], topn=5)
    n=1

    print("以下是推薦的歌曲:")
    for count, sim in sims:
        print(f'第{n}首  {corpusDF.Singer[count]}  {corpusDF.Title[count]}')
        print(f'{corpusDF.Lyric[count]}')
        print()
        print()
        n+=1

In [14]:
# 請在此輸入想要推薦的內容。

query=['請推薦我快樂的歌。']
recommand(query)


輸入段落：請 推薦 我 快樂 的 歌 。

以下是推薦的歌曲:
第1首  謝霆鋒  終點站
停下腳步 放棄追逐 請你被我照顧 畫個溫暖藍圖 愛的溫度 熱情恢復 手中的幸福值得傾訴 心事有你分攤不再沉重中走的太蹣跚 問號有了答案 你是我的港灣 心被感謝充滿 是你讓我感動用不完 我甘願為你不做單身漢 流浪到了終點站 快樂每天都瀏覽 還欠彼此心中的愛 一起不孤單 我們找到終點站 往下個起點追趕 有了你甘願平淡 牽著你變得勇敢 跟著你走一路上都平安 停下腳步 放棄追逐 請你被我照顧 畫個溫暖藍圖 愛的溫度 熱情恢復 手中的幸福值得傾訴 心事有你分攤不再沉重中走的太蹣跚 問號有了答案 你是我的港灣 心被感謝充滿 是你讓我感動用不完 我甘願為你不做單身漢 流浪到了終點站 快樂每天都瀏覽 還欠彼此心中的愛 一起不孤單 我們找到終點站 往下個起點追趕 有了你甘願平淡 牽著你變得勇敢 跟著你走一路上都平安 流浪到了終點站 快樂每天都瀏覽 還欠彼此心中的愛 一起不孤單 我們找到終點站 往下個起點追趕 有了你甘願平淡 牽著你變得勇敢 跟著你走一路上都平安


第2首  張杰  向前衝
電視劇 號手就位 主題曲 一提到 未來啊 我想過無數可能 會跨山 也追風 衝撞一段段旅程 頑劣小孩 長成可靠的大人 我沒想過 有一天戎裝一身 我仰望 的藍天 歸屬於我的靈魂 我雙手 的長劍 築起堅固的甲盾 午夜時分 總要有人守著燈 談起犧牲 是榮耀回歸了星塵 向前衝 我可以 不吝每一次演練當戰鬥 用十年 二十年 將夢百分百命中 我是號手 等東風起 要配得上光榮 我仰望 的藍天 它點亮我的靈魂 我雙手 的長劍 早在危難前動身 午夜時分 總要有人守著燈 談起犧牲 是榮耀回歸了星塵 向前衝 我可以 不吝每一次演練當戰鬥 用十年 二十年 將夢百分百命中 我的鐵拳英勇 每一擊有責任的厚重 我的國英雄 它的戰士向前衝 記住我們 名字叫號手 等東風起 那一刻光榮


第3首  黎明  相信
請相信我學會的堅持來自你的影響　有你在心中　困境能把我怎樣 每當我眼前山高水長　孤身為夢想好忙 你的一句關懷　把我每個明天都點亮 ＊我相信我看到的未來跟你一樣　困難只是暫時　我們會解開 　你教我怎麼駕馭情感　把思念熬成習慣 　相愛像穿越海洋　你像遠方那點星光 ＃因為愛　因為夢　因為相信所以堅定從容 　距離再遠　沒有埋怨　不需要誓言 △

In [13]:
# 請在此輸入想要推薦的內容。

query=['心情愉悅開開心心，春暖花開高興愉快。']
recommand(query)

輸入段落：心情 愉悅 開開 心心 ， 春暖花開 高興 愉快 。

以下是推薦的歌曲:
第1首  薛之謙  楚河漢界
你在楚河那面 灰色的天 沒有語言 分河界兩邊 天空留閃電 在破曉以前 你在河界那面 點起狼煙 手指向天 宣告你的野心 戰役不可避免 鐵騎粘著血液 一路往前 將字當先 要在天黑以前 奪走我的雙眼 你的軍隊過界 想隻手遮天 碰觸我的底線 軍令放左手邊 宣誓要將你殲滅 耶耶~ 我用兵果斷 功車馬連環 九宮能掀翻 你的千軍萬馬 請躺在我的城前 我用最最美的方式 解決你 我用最最美的佈陣 把你留在我的棋裡 那年群雄並起 想統一天下的你 我血快要留盡 雙手檔著你 就讓後人證明 用象牙刻成臉 刻你我英勇的野心 你的軍隊過界 想隻手遮天 碰觸我的底線 軍令放左手邊 宣誓要將你殲滅 耶耶~ 我用兵果斷 功車馬連環 九宮能掀翻 你的千軍萬馬 請躺在我的城前 我用最最美的方式 解決你 我用最最美的佈陣 把你留在我的棋裡 那年群雄並起 想統一天下的你 我血快要留盡 雙手檔著你 就讓後人證明 用象牙刻成臉 刻你我英勇的野心 那年群雄並起 想統一天下的你 我血快要留盡 雙手檔著你 就讓後人證明 用象牙刻成臉 刻你我英勇的野心 讓棋盤記住 我和你


第2首  王力宏  我就喜歡(勝利慶功版)
華人嘻哈　我就喜歡　　( ya c'mon　wu a ) 這是全新的嘻哈版本 Mc 嘻哈版本　我就喜歡　一個香芋派　謝謝 端端端端端　端端端端　端端端端 我就喜歡 端端端端端　端端端端　端端端端 我就喜歡　( uh uh　yo c'mon ) 我相信音樂真的不分什麼種族 西藏的戲曲變成最流行的音符 我們的嘻哈擁有黃顏色的皮膚 歡迎你一起加入我的Chink-out國度 誰看見龍的傳人黑色的眼珠都佩服 話不多　有點嚴肅　卻都有真功夫 天生就可以吃苦　永遠都不會認輸 世界的下一步　換我們做主 ＊ Ba la ba ba ba ba ba 　 Ba la ba ba ba ba ba　我就喜歡 　 Ba la ba ba ba ba ba 　 Ba la ba ba ba　我就喜歡　(什麼都不害怕) 　 Ba la ba ba ba ba ba 　 Ba la ba ba ba ba ba 　 Ba la ba　Ba la ba 　 Ba la ba ba ba　我就喜歡 來跳舞　我的小鼓　很特殊 